# Import necessary packages

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from random import sample

import re

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV

from timeit import default_timer as timer

from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [ ]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

**Note: This Step is for the extraction of California State and Foods category**

In [ ]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [ ]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1612,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,3,0,0,1,...,0,2,0,4,1,1,0,1,1,0
1613,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,0,1,0,1,...,1,3,1,0,0,1,2,0,0,0
1614,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,3,0,2,1,1,0,1,0,1,0
1615,FOODS_1_004_CA_1_validation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1616,FOODS_1_005_CA_1_validation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,3,9,3,3,...,3,1,1,2,0,2,2,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12191,FOODS_3_823_CA_4_validation,FOODS_3_823,FOODS_3,FOODS,CA_4,CA,1,1,1,4,...,3,1,2,2,0,1,1,3,3,0
12192,FOODS_3_824_CA_4_validation,FOODS_3_824,FOODS_3,FOODS,CA_4,CA,0,0,0,1,...,0,1,1,1,0,2,0,0,0,1
12193,FOODS_3_825_CA_4_validation,FOODS_3_825,FOODS_3,FOODS,CA_4,CA,2,3,2,1,...,0,2,0,1,3,1,1,0,2,0
12194,FOODS_3_826_CA_4_validation,FOODS_3_826,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,1,2,2,1,4,5,1,7,0,4


In [ ]:
INPUT_DIR_2 = r'/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/List of Product ID according to 4 demand patterns/California/Foods'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [ ]:
len(sales_intermittent)

1301

In [ ]:
len(sales_lumpy)

395

In [ ]:
len(sales_erratic)

458

In [ ]:
len(sales_smooth)

3594

**Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)**

In [ ]:
sales_intermittent

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1616,FOODS_1_005_CA_1_validation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,3,9,3,3,...,3,1,1,2,0,2,2,1,4,1
1625,FOODS_1_015_CA_1_validation,FOODS_1_015,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,1,1,0,2,0,1,2,2,0
1637,FOODS_1_027_CA_1_validation,FOODS_1_027,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,5,4,2,3,1,0,2,1,3,3
1638,FOODS_1_028_CA_1_validation,FOODS_1_028,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,1,5,4,2,1,1,1,3,0,0
1640,FOODS_1_030_CA_1_validation,FOODS_1_030,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,2,0,0,1,1,2,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12147,FOODS_3_779_CA_4_validation,FOODS_3_779,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12155,FOODS_3_787_CA_4_validation,FOODS_3_787,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,0,2,2,1,0,1,0,0,3
12168,FOODS_3_800_CA_4_validation,FOODS_3_800,FOODS_3,FOODS,CA_4,CA,1,2,4,2,...,0,3,2,3,1,1,2,1,2,2
12183,FOODS_3_815_CA_4_validation,FOODS_3_815,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,1,0,2,0,0,0,0,0,0


# User-defined functions to calculate Metrics

In [ ]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

# Format sales data and then merge with calendar data

In [ ]:
sales_pattern = sales_intermittent.copy()

In [ ]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

In [ ]:
dataframe_weekly

,wm_yr_wk,FOODS_1_005_CA_2,FOODS_1_005_CA_4,FOODS_1_008_CA_1,FOODS_1_008_CA_4,FOODS_1_009_CA_1,FOODS_1_009_CA_2,FOODS_1_009_CA_3,FOODS_1_009_CA_4,FOODS_1_010_CA_1,...,FOODS_3_821_CA_4,FOODS_3_823_CA_1,FOODS_3_823_CA_2,FOODS_3_823_CA_3,FOODS_3_823_CA_4,FOODS_3_824_CA_1,FOODS_3_824_CA_2,FOODS_3_824_CA_4,FOODS_3_825_CA_4,FOODS_3_826_CA_3
0,11101,36.0,4.0,0.0,1.0,4.0,6.0,6.0,5.0,0.0,...,0.0,0.0,24.0,18.0,2.0,2.0,3.0,1.0,6.0,0.0
1,11102,37.0,13.0,0.0,3.0,5.0,18.0,19.0,5.0,0.0,...,0.0,0.0,30.0,6.0,2.0,1.0,0.0,0.0,0.0,0.0
2,11103,40.0,3.0,0.0,3.0,0.0,8.0,4.0,2.0,0.0,...,0.0,16.0,8.0,31.0,20.0,0.0,0.0,0.0,0.0,0.0
3,11104,30.0,1.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,3.0,20.0,7.0,3.0,1.0,0.0,3.0,0.0
4,11105,33.0,8.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,38.0,6.0,0.0,1.0,2.0,5.0,16.0,6.0,3.0,...,0.0,0.0,17.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0
269,11609,19.0,13.0,0.0,1.0,4.0,18.0,9.0,0.0,0.0,...,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
270,11610,29.0,7.0,0.0,0.0,10.0,3.0,13.0,0.0,2.0,...,0.0,0.0,17.0,0.0,1.0,13.0,7.0,0.0,3.0,0.0
271,11611,43.0,7.0,0.0,2.0,5.0,6.0,7.0,3.0,0.0,...,0.0,0.0,31.0,2.0,2.0,2.0,1.0,1.0,1.0,0.0


In [ ]:
lags = list(range(1, 25))
moving_averages = list(range(1, 25)) 
stds = list(range(2, 25))
maximums = list(range(1, 25))
minimums = list(range(1, 25))
totals = list(range(1, 25))
num_days_zeros = list(range(1, 25))

correlation_threshold = 0.8

end_train_week = 11610

num_id_sample_for_tuning = 200

In [ ]:
sample_list = sample(list_pattern_py, num_id_sample_for_tuning)

In [ ]:
dataframe_check = dataframe_weekly[sample_list].iloc[-2:].T

dataframe_check['Check'] = np.where((dataframe_check[271] == 0) &
                                    (dataframe_check[272] == 0), 1, 0)

sample_removed = dataframe_check[dataframe_check['Check'] == 1].index.tolist()

In [ ]:
sample_list_filtered = [x for x in sample_list if (x not in sample_removed)]

In [ ]:
len(sample_list_filtered)

111

In [ ]:
start = timer()

df_output_hyper_params = pd.DataFrame()
    
for product in sample_list_filtered:
    print('Currently Running: %s' % product)
    print('Progressing: {0} %'.format(round(sample_list_filtered.index(product) / len(sample_list_filtered) * 100, 2)))

    dataframe_product = dataframe_weekly[['wm_yr_wk', product]]

    # Create Lag variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (t-{lag})': dataframe_product[product].shift(lag)
        for lag in lags
    })

    # Create Rolling Moving Average variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
        for moving_average in moving_averages
    })

    # Create Rolling Standard Deviation variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
        for std in stds
    })

    # Create Rolling Maximum variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
        for maximum in maximums
    })

    # Create Rolling Minimum variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
        for minimum in minimums
    })

    # Create Rolling Total variables 
    dataframe_product = dataframe_product.assign(**{
        f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
        for total in totals
    })


    dataframe_product.dropna(inplace=True)
    dataframe_product.set_index('wm_yr_wk', inplace=True)

    train_data = dataframe_product[dataframe_product.index <= end_train_week]

    corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
    high_corr_var=np.where(corr_matrix > correlation_threshold)
    high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]

    train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)

    X_train = train_data.drop(product, axis=1)
    y_train = train_data[product]  

    tscv = TimeSeriesSplit(n_splits=3)

    params = {'n_estimators': [30, 50],
              'max_depth': list(range(3,11)),
              'min_samples_leaf': range(4,11),
              'max_features': ['sqrt', 'log2', None],
              'bootstrap': [True], #We force bootstrap
              'max_samples': [0.6, 0.7, 0.8, 0.9]}

    forest = RandomForestRegressor(n_jobs=1)

    regressor = RandomizedSearchCV(estimator=forest,
                                  param_distributions=params,
                                  cv = tscv, 
                                  scoring='neg_mean_absolute_percentage_error',
                                  n_iter=150,
                                  verbose=1)
    regressor.fit(X_train, y_train)
    best_params_temp = regressor.best_params_

    df_output_temp = pd.DataFrame.from_dict(best_params_temp, orient='index').T
    df_output_temp['Product'] = product

    df_output_hyper_params = df_output_hyper_params.append(df_output_temp, ignore_index=True)

    
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: FOODS_3_478_CA_3
Progressing: 0.0 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_2_261_CA_1
Progressing: 0.9 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_3_497_CA_4
Progressing: 1.8 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_3_160_CA_1
Progressing: 2.7 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_3_342_CA_4
Progressing: 3.6 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_2_258_CA_4
Progressing: 4.5 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_3_657_CA_4
Progressing: 5.41 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_3_769_CA_1
Progressing: 6.31 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: FOODS_2_330_CA_2
Progressing: 7.21 

In [ ]:
df_output_hyper_params.to_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/Random Forest (Completed Pipeline)/Foods/Intermittent_Params_Tuning_Random Forest.csv')

# Decide optimum set of parameters for Random Forest algorithm

In [ ]:
# See if any values larger than 1 in column "Product"
# If yes, the corresponding set of parameters occur more than once and those sets should be considered first
df_output_hyper_params.groupby(['n_estimators', 'max_depth', 'min_samples_leaf', 'max_features', 'max_samples'])[['Product']].count().sort_values(by = 'Product', ascending=False)

Product
n_estimators max_depth min_samples_leaf max_features max_samples         
30           3         8                log2         0.8                1
             4         6                log2         0.8                1
                       7                log2         0.6                1
                       10               log2         0.8                1
             6         4                sqrt         0.9                1
             10        5                sqrt         0.6                1
                       7                sqrt         0.6                1
                       8                log2         0.9                1
50           4         9                log2         0.7                1
             9         10               log2         0.6                1

In [ ]:
df_output_hyper_params.groupby(['n_estimators'])[['Product']].count()

,Product
n_estimators,
30,81
50,30


In [ ]:
df_output_hyper_params.groupby(['max_depth'])[['Product']].count()

,Product
max_depth,
3,11
4,19
5,10
6,15
7,12
8,10
9,16
10,18


In [ ]:
df_output_hyper_params.groupby(['min_samples_leaf'])[['Product']].count()

,Product
min_samples_leaf,
4,19
5,21
6,12
7,15
8,12
9,16
10,16


In [ ]:
df_output_hyper_params.groupby(['max_features'])[['Product']].count()

,Product
max_features,
log2,7
sqrt,3


In [ ]:
df_output_hyper_params.groupby(['max_samples'])[['Product']].count()

,Product
max_samples,
0.6,24
0.7,23
0.8,28
0.9,36


# Implement Random Forest on Test Set with optimum set of parameters

### Specify optimum set of parameters

In [ ]:
n_estimators_chosen = 30
max_depth_chosen = 4
min_samples_leaf_chosen = 5
max_features_chosen = 'log2'
max_samples_chosen = 0.9

In [ ]:
len(list_pattern_py)

1301

In [ ]:
list_pattern_temp = list_pattern_py.copy()

In [ ]:
start = timer()

df_output_rf_pattern = pd.DataFrame()
for product in list_pattern_temp:
  print('Currently Running: %s' % product)
  print('Progressing: {0} %'.format(round(list_pattern_temp.index(product) / len(list_pattern_temp) * 100, 2)))
  dataframe_product = dataframe_weekly[['wm_yr_wk', product]]
  # Create Lag variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (t-{lag})': dataframe_product[product].shift(lag)
      for lag in lags
  })

  # Create Rolling Moving Average variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
      for moving_average in moving_averages
  })

  # Create Rolling Standard Deviation variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
      for std in stds
  })

  # Create Rolling Maximum variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
      for maximum in maximums
  })

  # Create Rolling Minimum variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
      for minimum in minimums
  })

  # Create Rolling Total variables 
  dataframe_product = dataframe_product.assign(**{
      f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
      for total in totals
  })

  dataframe_product.dropna(inplace=True)
  dataframe_product.set_index('wm_yr_wk', inplace=True)

  train_data = dataframe_product[dataframe_product.index <= end_train_week]
  test_data = dataframe_product[dataframe_product.index > end_train_week]

  corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
  high_corr_var=np.where(corr_matrix > correlation_threshold)
  high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]

  train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
  test_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)

  X_train = train_data.drop(product, axis=1)
  y_train = train_data[product]

  X_test = test_data.drop(product, axis=1)
  y_test = test_data[product]

  forest = RandomForestRegressor(n_jobs=1,
                                n_estimators = n_estimators_chosen,
                                min_samples_leaf = min_samples_leaf_chosen,
                                max_samples = max_samples_chosen,
                                max_features = max_features_chosen,
                                max_depth = max_depth_chosen,
                                bootstrap = [True],
                                random_state = 0)
  

  forest.fit(X_train,y_train)
  y_pred = forest.predict(X_test)
  df_output_temp = pd.DataFrame({'Actual Data': y_test, 
                                  'Forecast': y_pred, 
                                  'Product': [product for num_rows in range(len(y_pred))]
                                  })
  
  df_output_rf_pattern = df_output_rf_pattern.append(df_output_temp, ignore_index=False)  

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: FOODS_1_008_CA_1
Progressing: 0.0 %
Currently Running: FOODS_1_009_CA_1
Progressing: 0.08 %
Currently Running: FOODS_1_010_CA_1
Progressing: 0.15 %
Currently Running: FOODS_1_011_CA_1
Progressing: 0.23 %
Currently Running: FOODS_1_017_CA_1
Progressing: 0.31 %
Currently Running: FOODS_1_019_CA_1
Progressing: 0.38 %
Currently Running: FOODS_1_022_CA_1
Progressing: 0.46 %
Currently Running: FOODS_1_023_CA_1
Progressing: 0.54 %
Currently Running: FOODS_1_029_CA_1
Progressing: 0.61 %
Currently Running: FOODS_1_035_CA_1
Progressing: 0.69 %
Currently Running: FOODS_1_041_CA_1
Progressing: 0.77 %
Currently Running: FOODS_1_044_CA_1
Progressing: 0.85 %
Currently Running: FOODS_1_048_CA_1
Progressing: 0.92 %
Currently Running: FOODS_1_056_CA_1
Progressing: 1.0 %
Currently Running: FOODS_1_057_CA_1
Progressing: 1.08 %
Currently Running: FOODS_1_065_CA_1
Progressing: 1.15 %
Currently Running: FOODS_1_069_CA_1
Progressing: 1.23 %
Currently Running: FOODS_1_070_CA_1
Progressing: 1

In [ ]:
df_output_rf_pattern

,Actual Data,Forecast,Product
wm_yr_wk,,,
11611,0.0,0.198407,FOODS_1_008_CA_1
11612,0.0,0.198407,FOODS_1_008_CA_1
11611,5.0,5.285618,FOODS_1_009_CA_1
11612,7.0,5.004445,FOODS_1_009_CA_1
11611,0.0,1.345795,FOODS_1_010_CA_1
...,...,...,...
11612,0.0,1.441408,FOODS_3_823_CA_4
11611,1.0,0.583363,FOODS_3_824_CA_4
11612,0.0,1.229690,FOODS_3_824_CA_4


In [ ]:
df_output_rf_pattern.to_csv('Intermittent_Random_Forest_Forecast_Values.csv')

In [ ]:
# User-defined funcion to calculate metrics

def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [ ]:
# Start calculating metrics 
start = timer()

df_forecast_metrics = df_output_rf_pattern.groupby('Product').apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.03181664198333844 minutes


In [ ]:
df_forecast_metrics

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_005_CA_2,0.548193,35.4217,36.7390,30.8212,11.512061,13.941757
1,FOODS_1_005_CA_4,0.996745,44.2998,53.8180,41.1101,3.986981,4.512767
2,FOODS_1_008_CA_1,inf,inf,200.0000,NaN,0.198407,0.198407
3,FOODS_1_008_CA_4,0.649924,55.7078,65.7692,46.4625,1.949772,2.431010
4,FOODS_1_009_CA_1,0.570293,19.0098,19.4003,17.1101,1.140587,1.425451
...,...,...,...,...,...,...,...
1296,FOODS_3_824_CA_1,0.949307,189.8614,118.6212,45.7642,1.898614,2.138148
1297,FOODS_3_824_CA_2,2.462119,492.4238,154.7359,241.8509,2.462119,2.462505
1298,FOODS_3_824_CA_4,0.823164,164.6327,126.3134,41.6637,0.823164,0.918075
1299,FOODS_3_825_CA_4,inf,396.6287,132.5646,396.6287,3.966287,3.992288


In [ ]:
df_forecast_metrics.to_csv('Random_Forest_Intermittent_Forecast_Metrics.csv')